# **<span style="color:red">Imports</span>**

In [1]:
# Math
import math

# Handy arrays
import numpy as np

# DeepXDE
import deepxde as dde
import torch

Using backend: pytorch
Other supported backends: tensorflow.compat.v1, tensorflow, jax, paddle.
paddle supports more examples now and is recommended.


# <span style="color:red">**Постановка**</span>

<div>
<img src="var-07.png" width="600"/>
</div>

# <span style="color:red">**PINN**</span>

## <span style="color:orange">**Init**</span>

### <span>**Функция вычисления невязок**</span>

In [2]:
def pde(domain: torch.Tensor, u: torch.Tensor) -> torch.Tensor:
    # d^2 u
    # -----
    # dx ^2
    du_xx = dde.grad.hessian(u, domain, i=0, j=0)

    # d^2 u
    # -----
    # dy ^2
    du_yy = dde.grad.hessian(u, domain, i=1, j=1)

    # du/dt
    du_t = dde.grad.jacobian(u, domain, j=2)

    # Equation
    return ((du_t - 2.3 * (du_xx + du_yy))
            - 2.5586 * torch.exp(-0.7 * domain[:, 2:3]) * torch.sin(0.9 * domain[:, 0:1])
            - 13.5616 * torch.exp(-0.7 * domain[:, 2:3]) * torch.cos(2.2 * domain[:, 1:2]))

### <span>**Геометрия**</span>

In [3]:
# Boundaries
x_left, x_right = 0, math.pi
y_down, y_up = -math.pi, math.pi
T0, T1 = 0, 1

# X & Y geomentry (rectangle)
geometry = dde.geometry.Rectangle(xmin=[x_left, y_down], xmax=[x_right, y_up])
# Time segment [0, 1]
time_domain = dde.geometry.TimeDomain(T0, T1)
# Final domain
domain = dde.geometry.GeometryXTime(geometry, time_domain)

### <span>**Функции принадлежности точки границе**</span>

In [4]:
# Left boundary
def l_boundary(domain, on_boundary):
    return on_boundary and dde.utils.isclose(domain[0], x_left)

In [5]:
# Right boundary
def r_boundary(domain, on_boundary):
    return on_boundary and dde.utils.isclose(domain[0], x_right)

In [6]:
# Up boundary
def up_boundary(domain, on_boundary):
    return on_boundary and dde.utils.isclose(domain[1], y_up)

In [7]:
# Down boundary
def down_boundary(domain, on_boundary):
    return on_boundary and dde.utils.isclose(domain[1], y_down)

In [8]:
# Initial boundary
def initial_boundary(domain, on_initial):
    return on_initial and dde.utils.isclose(domain[2], T0)

### <span>**Константа и соответсвующие модели обучения**</span>

In [9]:
class TrainSchedule:
    def __init__(self, lr: float, iterations: int):
        self.lr = lr
        self.iterations = iterations

In [10]:
class ConstantPreset:
    def __init__(self, constant: float, train_schedules: list[TrainSchedule]):
        self.constant = constant
        self.train_schedules = train_schedules

In [410]:
# Different initial constant values and their train schedules
constant_sets = [ConstantPreset(6.5, [TrainSchedule(1e-3, 11000), TrainSchedule(1e-4, 5000)]),
                 ConstantPreset(5.0, [TrainSchedule(1e-3, 9000), TrainSchedule(1e-4, 2000)]),
                 ConstantPreset(-4.0, [TrainSchedule(1e-3, 10000), TrainSchedule(1e-4, 3000)])]

In [411]:
# Set selection
constant_set = constant_sets[2]

In [412]:
# Constant to train
c_lambda = dde.Variable(constant_set.constant)

### <span>**Граничные и начальные условия**</span>

In [413]:
# Boundary condition left
def bc_l_func(domain):
    if (isinstance(domain, torch.Tensor)):
        t = domain[:, 2:3]
    # Convert to numpy array to CUDA tensor
    else:
        t = torch.from_numpy(domain[:, 2:3]).to(device="cuda")

    return - 1.98 * torch.exp(-0.7 * t)

bc_l = dde.NeumannBC(domain, bc_l_func, l_boundary)

In [414]:
# Boundary condition right
def bc_r_func(domain):
    if (isinstance(domain, torch.Tensor)):
        y = domain[:, 1:2]
        t = domain[:, 2:3]
    # Convert to numpy array to CUDA tensor
    else:
        y = torch.from_numpy(domain[:, 1:2]).to(device="cuda")
        t = torch.from_numpy(domain[:, 2:3]).to(device="cuda")

    return 1.3 + 1.3 * torch.exp(-0.7 * t) * torch.cos(2.2 * y) - 0.55 * (1 - math.sqrt(5)) * torch.exp(-0.7 * t)

bc_r = dde.DirichletBC(domain, bc_r_func, r_boundary)

In [415]:
# Boundary condition up
def bc_up_func(domain):
    if (isinstance(domain, torch.Tensor)):
        t = domain[:, 2:3]
    # Convert to numpy array to CUDA tensor
    else:
        t = torch.from_numpy(domain[:, 2:3]).to(device="cuda")
        
    return - 2.86 * math.sqrt(0.625 - math.sqrt(5)/8) * torch.exp(-0.7 * t)

bc_up = dde.NeumannBC(domain, bc_up_func, up_boundary)

In [416]:
# Boundary condition down
def bc_down_func(domain):
    if (isinstance(domain, torch.Tensor)):
        x = domain[:, 0:1]
        t = domain[:, 2:3]
    # Convert to numpy array to CUDA tensor
    else:
        x = torch.from_numpy(domain[:, 0:1]).to(device="cuda")
        t = torch.from_numpy(domain[:, 2:3]).to(device="cuda")

    return 1.3 + 2.2 * torch.exp(-0.7 * t) * torch.sin(0.9 * x) + 0.875 * (1 + math.sqrt(5)) * torch.exp(-0.7 * t)

bc_down = dde.DirichletBC(domain, bc_down_func, down_boundary)

In [417]:
# Initial condition
def ic_func(domain):
    if (isinstance(domain, torch.Tensor)):
        x = domain[:, 0:1]
        y = domain[:, 1:2]
    # Convert to numpy array to CUDA tensor
    else:
        x = torch.from_numpy(domain[:, 0:1]).to(device="cuda")
        y = torch.from_numpy(domain[:, 1:2]).to(device="cuda")

    return c_lambda + 2.2 * torch.sin(0.9 * x) + 1.3 * torch.cos(2.2 * y)

ic = dde.IC(domain, ic_func, initial_boundary)

In [418]:
# Observed data
data = np.load('var-07.npz')
ob_domain = data['xyt']
ob_u = data['u']
observed_u = dde.icbc.PointSetBC(ob_domain, ob_u, component=0)

### <span>**Система уравнений**</span>

In [419]:
data = dde.data.TimePDE(domain, pde, [bc_l, bc_r, bc_up, bc_down, ic, observed_u],
                        num_domain=3000, num_boundary=1000, num_initial=2000,
                        anchors=ob_domain,
                        num_test=3000)

### <span>**Нейронная сеть**</span>

In [420]:
layer_size = [3] + [50] * 5 + [1]
net = dde.maps.FNN(layer_size, "tanh", "Glorot uniform")

### <span>**Обёртка нейронной сети**</span>

In [421]:
# Init model
model = dde.Model(data, net)

### <span>**Обучение**</span>

In [422]:
# Callbacks
variable = dde.callbacks.VariableValue(c_lambda, period=1000)
pde_resampler = dde.callbacks.PDEPointResampler(period=800)

In [ ]:
for train_schedule in constant_set.train_schedules:
    model.compile("adam", lr=train_schedule.lr, external_trainable_variables=[c_lambda])
    losshistory, trainstate = model.train(iterations=train_schedule.iterations, callbacks = [pde_resampler, variable])

In [ ]:
# Show training plot
dde.saveplot(losshistory, trainstate, issave=False, isplot=True)

In [ ]:
print(c_lambda)